In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import Bio
#from localcider.sequenceParameters import SequenceParameters
import scipy
from scipy.spatial.distance import cdist
import math

In [2]:
path = './data/pdbs'
from function import VdW_interaction
protein_test = VdW_interaction(path, 'AF-C0H3V8-F1.pdb')


SyntaxError: invalid syntax. Perhaps you forgot a comma? (function.py, line 108)

In [1]:
import numpy as np
from function import cluster_calc
array = np.array([
    [0, 10, 20, 30, 40, 50],
    [10, 0, 5, 0, 0, 0],
    [20, 5, 0, 0, 0, 0],
    [30, 0, 0, 0, 0, 0],
    [40, 0, 0, 0, 0, 0],
    [50, 0, 0, 0, 0, 0]
])

print(cluster_calc(array))

IndexError: column index (1) out of bounds

In [2]:
Cluster ={}
from scipy.sparse.csgraph import connected_components
from scipy.sparse import lil_matrix
n_atoms = len(array)
adjacency_matrix = lil_matrix((int(len(array)), int(len(array))))
for i in range(len(array)):
  adjacency_matrix[0,i] = array[0,i]
  adjacency_matrix[i,0] = array[i,0]
for i in range(1, len(array)):
  for j in range(i+1, len(array)):
    overlap_volume = array[i, j]
    adjacency_matrix[i, j] = overlap_volume

adjacency_matrix = adjacency_matrix.tocsr()
labels, n_components = connected_components(adjacency_matrix[1:,1:])

set_components = set(n_components) #unique number of clusters
list_components = n_components.tolist()

for i in range(len(set_components)):
  atom_number = []
  for n in range(len(list_components)):
    if list_components[n] == i:
      atom_number.append(adjacency_matrix[list_components[0],n+1])   
      Cluster[f'Cluster {str(i)}'] = atom_number

#print(adjacency_matrix)
#print(adjacency_matrix[0,0])
#print(list_components)
print(labels)
print(n_components)  
print(Cluster)

4
[0 0 1 2 3]
{'Cluster 0': [10.0, 20.0], 'Cluster 1': [30.0], 'Cluster 2': [40.0], 'Cluster 3': [50.0]}


In [31]:
array[0,1]

100

In [12]:
##Backup#######
def test_calc(array):
  Cluster ={}
  from scipy.sparse.csgraph import connected_components
  from scipy.sparse import lil_matrix
  n_atoms = len(array)
  adjacency_matrix = lil_matrix((int(len(array))-1, int(len(array))-1))
  for i in range(1, len(array)):
    for j in range(i+1, len(array)):
      overlap_volume = array[i, j]
      adjacency_matrix[i-1, j-1] = overlap_volume
  
  adjacency_matrix = adjacency_matrix.tocsr()
  labels, n_components = connected_components(adjacency_matrix, directed=False)
  
  set_components = set(n_components) #unique number of clusters
  list_components = n_components.tolist()

  for i in range(len(set_components)):
    atom_number = []
    for n in range(len(list_components)):
      if list_components[n] == i:
        atom_number.append(adjacency_matrix[list_components[0],n])   
        Cluster[f'Cluster {str(i)}'] = atom_number
  return Cluster
  
print(test_calc(numpy_array))  

{'Cluster 0': [0.0], 'Cluster 1': [0.0], 'Cluster 2': [0.0], 'Cluster 3': [0.0], 'Cluster 4': [0.0]}
